In [37]:
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [38]:
# 加载Iris数据集
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 数据预处理：标准化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 将数据集拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 将数据转换为Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# 创建DataLoader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)


In [39]:
# 定义一个简单的神经网络
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(4, 16)  # 输入4个特征
        self.fc2 = nn.Linear(16, 3)  # 输出3个类别（Setosa, Versicolor, Virginica）

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [40]:
# 模型、损失函数和优化器
model = SimpleModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
# 定义warm-up的策略：前5个epoch，学习率逐渐从0增大到初始值0.001
def warm_up(epoch):
    warm_up_epochs = 5
    if epoch < warm_up_epochs:
        return 10 * float(epoch + 1) / float(warm_up_epochs)
    return 1.0

In [42]:
# 训练循环
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        
        # 计算损失
        loss = criterion(outputs, targets)
        loss.backward()
        
        # 更新权重
        optimizer.step()
    
    # 输出当前epoch和损失
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/20], Loss: 0.6306
Epoch [2/20], Loss: 0.4714
Epoch [3/20], Loss: 0.5722
Epoch [4/20], Loss: 0.5641
Epoch [5/20], Loss: 0.5238
Epoch [6/20], Loss: 0.6420
Epoch [7/20], Loss: 0.5159
Epoch [8/20], Loss: 0.7078
Epoch [9/20], Loss: 0.3290
Epoch [10/20], Loss: 0.4055
Epoch [11/20], Loss: 0.5817
Epoch [12/20], Loss: 0.6311
Epoch [13/20], Loss: 0.4636
Epoch [14/20], Loss: 0.5794
Epoch [15/20], Loss: 0.4240
Epoch [16/20], Loss: 0.3031
Epoch [17/20], Loss: 0.2813
Epoch [18/20], Loss: 0.6013
Epoch [19/20], Loss: 0.2927
Epoch [20/20], Loss: 0.3778
